## Overview of Key Results so Far

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In this research project, we are interested in knowing how the future southern ocean carbon sink will be impacted by the interaction of 2 main drivers:

- evolution of atmospheric O$_3$ concentration in the Southern Ocean 
    - 3 different scenarios:
    - no ozone hole ("Fixed Ozone" or "Ozone 1950")
    - ozone hole opens and then recovers ("Hist. Ozone")
    - ozone hole opens around 1990 and then never recovers ("1990 Ozone")
- evolution of atmospheric GHG concentrations
    - 2 scenarios: 
    - SSP1-2.6 (LOW CARBON)
    - SSP 3-7.0 (HIGH CARBON)
    
We force a model (currently PlankTOM12, with a small adaptation to prevent GOC "explosions", see https://nbviewer.org/github/tjarnikova/SOZONE/blob/master/MODPROC_ROBOT/MultiModelMonitor/SANITY_MONITOR.ipynb) with atmospheric forcing from 6 scenarios (combination of the above) of the UKESM1. This forcing has been adapted to work with NEMO (see https://nbviewer.org/github/tjarnikova/SOZONE/tree/master/MO_pipeline/ and https://docs.google.com/document/d/1sK9lCSdDUcnPzPrVHjrkluf2xh09VzTUMCh7yDlOkDk/edit). 






In [4]:
pdir = '../PICTURES/OVERVIEW/wspd_10yrmean.jpg'
from IPython.display import Image
Image(url= pdir, width=700, height=400)

In [ ]:
from IPython.display import Image
  
# get the image
Image(url="snowfall4.jpeg", width=300, height=300)